<h1><center>Lottery gamifier</center></h1>

### Methodology
* Choose random retailers meeting certain criteria weighted based on the purchasing ability of retailers (previous day order is considered as purchasing ability)
* Choose the number of random retailers based on the weightage of number of retailers under each AM & retail group
* Choose the number of retailers (one fourth) of total retailers in a day

***

In [1]:
# Numpy & Pandas for calcs & dataframe
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)

# Random pick
import random

# Azure data & credentials (custom package for SQL)
from ckpackages import azsql

# Move files to folder
import shutil
import os,time
import datetime
import datetime as dt

# Merge csv into one
from glob import glob

# IPY - VOILA 
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import widgets,Button, Output, VBox,interactive,interact,IntProgress
import time
from IPython.display import HTML
import base64

# MAIL
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

from datetime import datetime
from os import path, remove
import configparser
import pandas as pd
import shutil
from os import path

In [5]:
## Move last day results to history
path = os.getcwd()

now = dt.datetime.now()
ago = now-dt.timedelta(hours=48)
strftime = "%H:%M %m/%d/%Y"
created = os.path.join(path + '\Results_csv')
dest = os.path.join(path + '\History_csv')

# Detect files from one folder created or modified in the past 24 hours
for root, dirs,files in os.walk(created):  
    for fname in files:
        path = os.path.join(root, fname)
        st = os.stat(path)    
        mtime = dt.datetime.fromtimestamp(st.st_mtime)
        if mtime > ago:
            print("True:  ", fname, " at ", mtime.strftime("%H:%M %m/%d/%Y"))
            shutil.move(path, dest)
            # this is actual move

In [6]:
## Read & join csv in same folder
a = os.getcwd()
path = a + '\\History_csv'
all_files = glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
df_concat = pd.concat(li, axis=0, ignore_index=True)

# Convert InvDate to Date format
df_concat['InvDate'] = pd.to_datetime(df_concat['InvDate'])

## Filter data by instances occuring in last week
days=7 #Last 6 days from current date
cutoff_date = df_concat['InvDate'].max() - pd.Timedelta(days=days)
max_date = df_concat['InvDate'].max()
#print(cutoff_date)

df1_concat = df_concat[(df_concat['InvDate'] >= cutoff_date)]
df1_concat = df_concat[(df_concat['InvDate'] < max_date)]

# List of last 6 days unique retailers
rtr_last6days = df1_concat[['RtrCode','RtrName']].drop_duplicates()
rtrlist_last6days = rtr_last6days['RtrCode'].tolist()
rtrnames_last6days = rtr_last6days['RtrName'].tolist()

# Filter retailer occuring in last 6 days (to be included below)
#data_filtered = data[~data.RtrCode.isin(rtr_last6days)]

In [57]:
print('\nList of retailers selected in last week:')
print(rtrnames_last6days)

['Arul murugan store', 'JAIBAIRAVI TRADERS', 'SRI MURUGAN SUPER MARKET', 'AKSHYA TRADERS', 'R.C.TRADERS', 'G.v traders ', 'VANAJA STORES ', 'NEW BALAJI STORE.', 'ANNAMALAI  TRADERS', 'ARUL  MURUGAN   STORE', 'Lakshmi MALLIGAI', 'Bhaju store', 'Mohammed zain', 'National Store', 'SRI KANNA STORE', 'VISVAAM STORE', 'JAYAMURUGAN STORE', 'SARAVANA STORE', 'GREEN BAZAAR ', 'SAIYED ARABHADH ', 'MATHINA STORE', 'SIVA RICE MALLIGAI ', 'Murugan Stores Super Market', 'Stmmaligai', 'Srivigneswarastore', 'Matha store', 'Iruthay meri', 'Welcome stores', 'Anitha stores', 'NEW NELLAI STORR', 'Pasumai supermarket', 'SRI JAYAM MALIGAI STORE', 'BILLAL MALIGAI ', 'SRI KRISHNA SUPER MARKET', 'S.R VETRI STORE', 'APPLE SHOPPING MALL', 'LAKSHMI BANGLES', 'ARUNA AGENCIES', 'm.h yosuf store']


In [30]:
## Import data
Query = 'select * from v_PC_MeeraRetailerCampaign' 
data = azsql.callstatement(Query)

# Add weekday & current weekday
data['weekday'] = data['InvDate'].dt.dayofweek
currentWeekDay = dt.datetime.now().weekday()

# Data only for Latest date
# On monday, get Sat & Sun data, for other days, get previous day data
if currentWeekDay==0:
    cutoff_date = data['InvDate'].max() - pd.Timedelta(days=1)
    max_date = data['InvDate'].max()
    
    data = data[(data['InvDate'] >= cutoff_date)]
    data = data[(data['InvDate'] <= max_date)]
else:
    data = data[data['InvDate'] == data['InvDate'].max()]

## Remove null - Retailers without AM
# Consider Data with AMName ( remove nulls)
data = data[data['AMName'].notnull()]

# Categories retail group as Wholesale & Retail
data['RtrGroupCde1'] = data['RtrGroupCde'].apply(lambda x: 'PC-WHOLESALE' if x=='PC-WHOLESALE' else 'PC-RETAIL')

# Filter retailer occuring in last 6 days
data = data[~data.RtrCode.isin(rtr_last6days)]

InvDate =  data['InvDate'].iloc[0]
InvDate = str(InvDate)[0:10]

maxdate= str(data['InvDate'].max())[0:10]
print('\nDate: ',maxdate)

TotalRtrCount = data['RtrCode'].count()
print('Unique retailers:', TotalRtrCount)

## Retailer master
rtr_master = data[['RtrCode','RtrName','SMSysName','DistName','RtrGroupCde1','RtrGroupCde','InvDate','PolypouchCount']].drop_duplicates()
rtr_master['PolypouchCount'] = rtr_master['PolypouchCount'].astype(int)

AM_RTR_in = data.groupby(['AMName','RtrGroupCde1'])['RtrCode'].count().reset_index()

# Create text widget for output
output_slider_variable = widgets.Text()



Date:  2020-10-07
Unique retailers: 318

Choose number of Lots


interactive(children=(IntSlider(value=55, description='Lots', min=10), Output()), _dom_classes=('widget-intera…

In [ ]:
# Define function to bind value of the input to the output variable 
def f(Lots):
    output_slider_variable.value = str(Lots)

print('\nChoose number of Lots')
# Create input slider with default value = 10  
interact(f, Lots=(10, 300, 5))

def picks(output_slider_variable,data):
    # Create and output new int variable with value of slider
    new_variable = int(output_slider_variable.value)

    ## Choose number of retailers to choose
    ## Weightage for AM
    # Percentage of Retailers across AM-Wise        
    TotalRtrCount = data['RtrCode'].count()
    AM_RTR_WGT = data.groupby(['AMName','RtrGroupCde1'])['RtrCode'].count().reset_index()
    AM_RTR_WGT['LotsPerAM'] = np.ceil((AM_RTR_WGT['RtrCode']/TotalRtrCount) * new_variable)


    ## Split dataframe on AM's
    # Select required rows
    data1 = data[['ZM','AMName','RtrGroupCde1','RtrCode','RtrName','TotalQty','PolypouchCount']]
    data2 = [rows for _, rows in data1.groupby(['AMName','RtrGroupCde1'])]
    return data2,AM_RTR_WGT,new_variable
    
## Function to generate random Rtr's with AM weightage
def randomRtrAmWise(i):
    data3,AM_RTR_WGT,lots = picks(output_slider_variable,data)
    # Retailer name
    a = data3[i]['RtrCode'].tolist()

    # Polypouch count as weight
    ppc = data3[i]['PolypouchCount'].tolist() 

    c=int(AM_RTR_WGT['LotsPerAM'][i])
    #random.shuffle(a)
    l = random.choices(a, weights=ppc, k=c)
    l.sort()
    df_R = pd.DataFrame(l,columns = ['RtrCode'])
    df_R['AMName'] = AM_RTR_WGT['AMName'][i]
    return df_R,AM_RTR_WGT,lots
    
def randompick(rtr_master,AM_RTR_in):
    df_RtrRP = pd.DataFrame()
    for j in range(len( AM_RTR_in.index.tolist())):
        df,AM_RTR_WGT,lots = randomRtrAmWise(j)
        df_RtrRP = df_RtrRP.append(df)

    df_RtrRP = df_RtrRP.merge(rtr_master,on='RtrCode')
    df_RtrRP = df_RtrRP.groupby('RtrCode').head(2).reset_index(drop=True) # Keep 1st two instances of duplicates

    index = df_RtrRP.reset_index().index
    number_of_rows = len(index)
    return df_RtrRP,lots,number_of_rows,AM_RTR_WGT

def randompickAM(rtr_master,AM_RTR_in):
    result,lots,number_of_rows,AM_RTR_WGT = randompick(rtr_master,AM_RTR_in)
    while (number_of_rows!=lots):
        if (number_of_rows==lots):
            display(result)
            print('Ok')
        else:
            result,lots,number_of_rows,AM_RTR_WGT = randompickAM(rtr_master,AM_RTR_in)
            break
    return result,lots,number_of_rows,AM_RTR_WGT


def create_download_link( df, title = "Download CSV file", filename = "results.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [ ]:
print('\n')
# Button pick
button = Button(description='Pick')

out = Output()


def on_click(_):
    with out:
        clear_output(True)
        dr,lots,number_of_rows,AM_RTR_WGT = randompickAM(rtr_master,AM_RTR_in)
        
        max_count = 100

        f = IntProgress(min=0, max=max_count) # instantiate the bar
        display(f) # display the bar

        count = 0
        while count <= max_count:
            f.value += 1 # signal to increment the progress bar
            time.sleep(.1)
            count += 1
                
        print('Count of lots: ',lots)
        print('Count of picked retailers: ',number_of_rows)
        print('\n Split of Retailers chosen acorss AMs')
        display(AM_RTR_WGT)
        
        print('\nThe selected retailers on date', InvDate)
        display(dr)
        
        # Save results
        root = 'Results_csv' 
        sheet = 'results_' + InvDate + '.csv' 
        file_name = os.path.join(root,sheet)
        print(file_name)
        dr.to_csv(file_name,index=False)
        create_download_link(dr)


button.on_click(on_click)

VBox([button])

***

In [ ]:
VBox([out])

In [ ]:
def send_mail():
    import os 
    path = os.getcwd()
    created1 = os.path.join(path + '\Results_csv')

    for root, dirs,files in os.walk(created1):  
        for fname in files:
            FOLDER = os.path.join(root, fname)
            a = fname

    FOLDER = os.path.join(created1,a)

    msg = MIMEMultipart()
    fromaddr = '<...from mail...>'
    msg['From'] = fromaddr
    msg["To"] = "<...to mail...>"
    #msg["Cc"] = "yourEmail@outlook.com"
    msg['Subject'] = "Retailer Offer Diwali 2020 - " + a
    
    htmlEmail = """
    <p><br/><br/>
        Please find the attached Report below.<br/><br/>
    <br/></p>
        Thank you! <br/><br/>
        Best Regards <br/>
         </p>
    <br/><br/>
    <font color="red">Please do not reply to this email as it is auto-generated. </font>
    """

    with open(FOLDER, "rb") as attached_file:
            part = MIMEApplication(
                attached_file.read(),
                Name=a)         

    part.add_header('Content-Disposition', "attachment; filename= %s" % a)
    # attach the instance 'p' to instance 'msg'
    msg.attach(part)

    msg.attach(MIMEText(htmlEmail, 'html'))
    server = smtplib.SMTP('smtp.office365.com', 587)
    server.starttls()
    server.login(fromaddr, "<...password...>")
    text = msg.as_string()
    server.sendmail(fromaddr, ['<...mail...>'], text)
    server.quit()

print('\n')
# Button pick
button1 = Button(description='Send Email')
    
out1 = Output()
    
def on_click1(_):
    with out1:
        clear_output(True)
        send_mail()        
        max_count = 100

        f = IntProgress(min=0, max=max_count) # instantiate the bar
        display(f) # display the bar

        count = 0
        while count <= max_count:
            f.value += 1 # signal to increment the progress bar
            time.sleep(.1)
            count += 1
            
        print("Email sent successfully!")

button1.on_click(on_click1)

VBox([button1,out1])